# Some thoughts on software design

As an example, consider this data.

In [1]:
import json

def parse_line(line):
    try:
        d = json.loads(line)
    except json.JSONDecodeError:
        return None
    return d['caption']

data = list(filter(None, (parse_line(row) for row in open('captions.txt', 'r'))))
data[:10]

[' Les Lieberman, Barri Lieberman, Isabel Kallman, Trish Iervolino, and Ron Iervolino ',
 ' Chuck Grodin ',
 ' Diana Rosario, Ali Sussman, Sarah Boll, Jen Zaleski, Alysse Brennan, and Lindsay Macbeth ',
 ' Kelly Murro and Tom Murro ',
 ' Ron Iervolino, Trish Iervolino, Russ Middleton, and Lisa Middleton ',
 ' Barbara Loughlin, Dr Gerald Loughlin, and Debbie Gelston ',
 ' Julianne Michelle ',
 ' Heather Robinson, Kiwan Nichols, Jimmy Nichols, Melanie Carbone, and Nancy Brown ',
 ' Bill Mack and Les Lieberman ',
 ' Dr David Lyden and Patricia Sorenson ']

**Goal:** Find the top 10 people by number of unique contacts

## Bottom-up design

Start by writing the little pieces that you know you need

Assemble the low level pieces into more complicated functions

&vellip;

Assemble a full solution

## Top-down design

Write the top-level function, put placeholders for the pieces it needs

Then write one of the placeholders, inserting placeholders as needed

&vellip;

Write the low-level elements

In [ ]:
def find_top_n(captions, n=10):
    connections = get_connections(captions)
    sorted_conn = sort_by_conn_number(connections)
    return sorted_conn[:n]

In [19]:
from collections import defaultdict

In [71]:
def get_connections(captions):
    """
    Take a list of captions as input, returns a connections structure.
    
    Input:
        captions   List of strings of captions
        
    Return:        Dictionary mapping names to a set of other names
    """
    
    retval = defaultdict(set)
    for caption in captions:
        names = get_names(caption)
        conns = generate_connections(names)
        for name, conn in conns.items():
            retval[name].update(conn)
    return retval

In [41]:
def get_names(caption):
    """Takes a string, return a list of strings (the names)"""
    names = caption.replace(' and ', ',').split(',')
    cleaned = filter(None, [n.strip() for n in names])
    return list(cleaned)

In [30]:
data[0].replace('and', ',')

' Les Lieberman, Barri Lieberman, Isabel Kallman, Trish Iervolino, , Ron Iervolino '

In [44]:
get_names(data[3])

['Kelly Murro', 'Tom Murro']

In [42]:
get_names('Sandy and Kevin')

['Sandy', 'Kevin']

In [47]:
[(d, get_names(d)) for d in data[:10]]

[(' Les Lieberman, Barri Lieberman, Isabel Kallman, Trish Iervolino, and Ron Iervolino ',
  ['Les Lieberman',
   'Barri Lieberman',
   'Isabel Kallman',
   'Trish Iervolino',
   'Ron Iervolino']),
 (' Chuck Grodin ', ['Chuck Grodin']),
 (' Diana Rosario, Ali Sussman, Sarah Boll, Jen Zaleski, Alysse Brennan, and Lindsay Macbeth ',
  ['Diana Rosario',
   'Ali Sussman',
   'Sarah Boll',
   'Jen Zaleski',
   'Alysse Brennan',
   'Lindsay Macbeth']),
 (' Kelly Murro and Tom Murro ', ['Kelly Murro', 'Tom Murro']),
 (' Ron Iervolino, Trish Iervolino, Russ Middleton, and Lisa Middleton ',
  ['Ron Iervolino', 'Trish Iervolino', 'Russ Middleton', 'Lisa Middleton']),
 (' Barbara Loughlin, Dr Gerald Loughlin, and Debbie Gelston ',
  ['Barbara Loughlin', 'Dr Gerald Loughlin', 'Debbie Gelston']),
 (' Julianne Michelle ', ['Julianne Michelle']),
 (' Heather Robinson, Kiwan Nichols, Jimmy Nichols, Melanie Carbone, and Nancy Brown ',
  ['Heather Robinson',
   'Kiwan Nichols',
   'Jimmy Nichols',
   'Me

In [35]:
if '':
    print('yes')
else:
    print('no')

no


In [53]:
s = set([1,2,3])

In [55]:
s.remove(1)

In [56]:
s

{2, 3}

In [68]:
def generate_connections(names):
    """Takes a list of names, returns a dictions mapping names to sets of names.
    
    For example: ["Joe", "Elizabeth"] -> {'Joe': {'Elizabeth'}, 'Elizabeth': {'Joe'}}
    """
    
    if len(names) < 2:
        return dict()
    
    retval = defaultdict(set)
    for name in names:
        all_names = set(names)
        all_names.remove(name)
        retval[name] = all_names
    return retval

In [69]:
assert generate_connections(["Joe", "Elizabeth"]) == {'Joe': {'Elizabeth'}, 'Elizabeth': {'Joe'}}

assert generate_connections(["Joe"]) == {}

In [60]:
generate_connections(["Joe", "Elizabeth"])

defaultdict(set, {'Joe': None, 'Elizabeth': None})

In [70]:
generate_connections(["Joe"])

{}

In [67]:
{}

{}

In [72]:
get_connections(data[:10])

defaultdict(set,
            {'Les Lieberman': {'Barri Lieberman',
              'Bill Mack',
              'Isabel Kallman',
              'Ron Iervolino',
              'Trish Iervolino'},
             'Barri Lieberman': {'Isabel Kallman',
              'Les Lieberman',
              'Ron Iervolino',
              'Trish Iervolino'},
             'Isabel Kallman': {'Barri Lieberman',
              'Les Lieberman',
              'Ron Iervolino',
              'Trish Iervolino'},
             'Trish Iervolino': {'Barri Lieberman',
              'Isabel Kallman',
              'Les Lieberman',
              'Lisa Middleton',
              'Ron Iervolino',
              'Russ Middleton'},
             'Ron Iervolino': {'Barri Lieberman',
              'Isabel Kallman',
              'Les Lieberman',
              'Lisa Middleton',
              'Russ Middleton',
              'Trish Iervolino'},
             'Diana Rosario': {'Ali Sussman',
              'Alysse Brennan',
              '

In [ ]:
def sort_by_conn_number(connections):
    """
    Takes a connection structure and return names sorted by unique connections
    
    Input:
        connections  Dictionary mapping names to sets of names
        
    Return:          A list of names ordered by number of connections, starting with most connections
    """